In [1]:
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.base import \
    BaseEstimator, RegressorMixin
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import numpy as np
from math import ceil
from collections import Counter
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

***Cross-validation***

In [2]:
def crossValidation(self, X, y, folds=10):
  acuracia = {'global': [], 'class': []}
  p_tamanho = X.shape[0] / folds
  
  normalizer = MinMax()

  for n in range(folds):
      inicio = int(n * p_tamanho)
      fim = int((n + 1) * p_tamanho)

      X_treino = normalizer.fit_transform(np.concatenate([X[:inicio], X[fim:]]))
      y_treino = np.concatenate([y[:inicio], y[fim:]])

      X_valid = normalizer.fit_transform(X[inicio:fim])
      y_valid = y[inicio:fim]

      self.fit(X_treino, y_treino)

      y_chapeu = self.predict(X_valid)
      c_matriz = confusion_matrix(y_valid, y_chapeu)

      accuracia = c_matriz.diagonal().sum() / c_matriz.sum()
      acuracia['global'].append(accuracia)

      accuraciaClass = c_matriz.diagonal() / c_matriz.sum(axis=1)
      acuracia['class'].append(accuraciaClass)
      
      print("##############################################################")
      print("Média da acurácia global:", np.mean(acuracia["global"]))
      print("Desvio Padrão global:", np.std(acuracia["global"]))
      
      print("\n")
      
      print("Média da acurácia por classe:", np.mean(acuracia["class"], axis=0))
      print("Desvio padrão por classe:", np.std(acuracia["class"], axis=0))
      print("##############################################################")

      print("\n")
      
  return acuracia

***Funções utilitárias***

In [3]:
class MinMax():
  def fit_transform(self, X):
      self.min = X.min(axis=0)
      self.max = X.max(axis=0)
      
      return (X - self.min)/(self.max - self.min)

def encode(count, label):
  encoded = np.zeros(count)
  encoded[int(label)] = 1
  return encoded

def label_encoder(y):
  count = int(y.max() + 1)
  return np.array([ encode(count, label) for label in y ])
  
def label_decoder(Y):  
  return Y.argmax(axis=1)

***QUESTÃO 01*** - breastcancer.csv

**ITEM (A)**

***Regressão Logística*** 

(Treinado com GD)

In [4]:
class LogisticRegression:
  def __init__(self, alpha=0.01, epochs=1000):
    self.alpha = alpha
    self.epochs = epochs

  def GD(self):
    n, m = self.X_biased.shape
    n_class = self.Y.shape[1]
    W = np.zeros((m, n_class))
    Y_hat = np.zeros((n, n_class))
    for _ in range(self.epochs):
      for k in range(n_class):
        y_sum = np.exp(self.X_biased @ W).sum(axis=1)
        y_hat = np.exp(self.X_biased @ W[:,k])
        e = (self.Y[:,k] - y_hat / y_sum).reshape(-1, 1)
        W[:,k] = W[:,k] + self.alpha * (1 / n) * (e * self.X_biased).sum(axis=0)
    self.W = W    
    
  def add_bias(self, X):
    n = X.shape[0]
    return np.c_[np.ones(n), X]

  def fit(self, X, y):
    self.X = X
    self.y = y
    self.X_min = X.min(axis=0)
    self.X_max = X.max(axis=0)
    self.X_biased = self.add_bias(self.X)
    self.Y = label_encoder(y)
    self.GD()

  def predict(self, X):
    X_biased = self.add_bias(X)
    Y = X_biased @ self.W
    return label_decoder(Y)

  def activation(z):
    return 1 / (1 + np.exp(-z))

***Análise do Discriminante Gaussiano***

In [5]:
class GaussianDiscriminantAnalysis():
  def fit(self, X, y):
    n = X.shape[0]
    self.X = X
    self.y = y
    self.covs = []
    self.means = []
    self.probs = []
    self.n_classes = int(y.max() + 1)
    data = np.c_[ X, y ]
    for k in range(self.n_classes):
      data_k = data[data[:,-1] == k]
      X_k = data_k[:,:-1]
      self.probs.append(data_k.shape[0] / n)
      self.covs.append(np.cov(X_k.T))
      self.means.append(X_k.mean(axis=0))
  
  def predict_single(self, x):
    class_predictions = []
    for k in range(self.n_classes):
      cov_det = np.linalg.det(self.covs[k])
      cov_inv = np.linalg.inv(self.covs[k])
      prediction = np.log(self.probs[k]) - 0.5 * np.log(cov_det) - 0.5 * (x - self.means[k]).T @ cov_inv @ (x - self.means[k])
      class_predictions.append(prediction)
    return np.argmax(class_predictions)

  def predict(self, X):
    return np.apply_along_axis(self.predict_single, 1, X)

***Naive Bayes Gaussiano***

In [6]:
class GaussianNaiveBayes():
  def fit(self, X, y):
    self.m = X.shape[1]
    n = X.shape[0]
    self.X = X
    self.y = y
    self.stds = []
    self.means = []
    self.probs = []
    self.n_classes = int(y.max() + 1)
    data = np.c_[ X, y ]
    for k in range(self.n_classes):
      data_k = data[data[:,-1] == k]
      X_k = data_k[:,:-1]
      self.probs.append(data_k.shape[0] / n)
      self.stds.append(X_k.std(axis=0))
      self.means.append(X_k.mean(axis=0))
  
  def predict_single(self, x):
    class_predictions = []
    for k in range(self.n_classes):
      sum = 0
      for d in range(self.m):
        var = self.stds[k][d] ** 2
        sum += np.log(2 * np.pi * var) + (x[d] - self.means[k][d]) ** 2 / var
      prediction = np.log(self.probs[k]) - 0.5 * sum
      class_predictions.append(prediction)
    return np.argmax(class_predictions)

  def predict(self, X):
    return np.apply_along_axis(self.predict_single, 1, X)

**ITEM (B)**

Para cada modelo criado, reporte valor médio e desvio padrão da acurácia
global e da acurácia por classe

***Importando os Dados***

In [7]:
df = np.genfromtxt("breastcancer.csv", delimiter=",")
X_b = df[:,:-1]
y_b = df[:,-1]

#print (X)
#print (y)

FileNotFoundError: ignored

***Regressão Logística - (Treinado com GD)***

In [ ]:
self = LogisticRegression()
print("Resultado para a regressão logística - (treinado com GD)\n")

resultado = crossValidation(self, X_b, y_b)

***Análise do Descriminante Gaussiano***

In [ ]:
self = GaussianDiscriminantAnalysis()
print("Resultado para a análise do descriminante gaussiano\n")

resultado = crossValidation(self, X_b, y_b)

***Naive Bayes Gaussiano***

In [ ]:
self = GaussianNaiveBayes()
print("Resultado para o naive bayes gaussiano\n")

resultado = crossValidation(self, X_b, y_b)

***QUESTÃO 02*** - vehicle.csv

**ITEM (A)**

***Regressão Softmax***

Treinado com GD

In [ ]:
class SoftmaxRegression():
  def __init__(self, alpha=0.01, epocs=1000):
    self.epocs = epocs
    self.alpha = alpha

  def fit(self, X, y):
    self.classes, self.counts = np.unique(y, return_counts=True)
    self.bias = np.zeros(self.classes.shape[0])
    self.weights = np.zeros((self.classes.shape[0], X.shape[1]))
    
    binarizer = sklearn.preprocessing.LabelBinarizer()
    binarizer.fit(range(self.classes.size))
    y_ = binarizer.transform(y)
    
    for i in range(self.epocs):
        parts = self.activation(X)
        for k, c in enumerate(self.classes):
            y_chapeu = parts[k]/parts.sum(axis=0)
            err_k = y_chapeu - y_[:, k:k+1]
            g_bias = err_k.mean()
            g_weights = X.T.dot(err_k).mean(axis=1)
            
            self.bias[k] -= self.alpha * g_bias
            self.weights[k] -= self.alpha * g_weights
              
  def predict(self, X):
    y_chapeu = np.matmul(X, self.weights.T) + self.bias  
    return np.argmax(y_chapeu, axis=1)

  def activation(self, X):
    parts = []
    for i, c in enumerate(self.classes):
        parts.append(np.exp(self.weights[i].T * X + self.bias[i]))

    return np.array(parts)

**ITEM (B)**

Para cada modelo criado, reporte valor médio e desvio padrão da acurácia global e da acurácia por classe

***Importando os Dados***

In [ ]:
df = np.genfromtxt("vehicle.csv", delimiter=",")
X_v = df[:,:-1]
y_v = df[:,-1]

***Regressão Softmax (Treinado com GD)***

In [ ]:
self = SoftmaxRegression()
print("Resultado para a regressão softmax\n")

resultado = crossValidation(self, X_v, y_v)

***Análise do Descriminante Gaussiano***

In [ ]:
self = GaussianDiscriminantAnalysis()
print("Resultado para a análise do descriminante gaussiano\n")

resultado = crossValidation(self, X_v, y_v)

***Naive Bayes Gaussiano***

In [ ]:
self = GaussianNaiveBayes()
print("Resultado para o naive bayes gaussiano\n")

resultado = crossValidation(self, X_v, y_v)